In [1]:
import os
from glob import glob
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format
from functools import reduce
from reports import create_confusion_matrix, create_classification_report, average_classification_report, sum_confusion_matrix

In [2]:
result_dir = './results'

target_dirs = sorted([item for item in glob(os.path.join(result_dir, '*')) if os.path.isdir(item)])
report_filepath_format = 'classification_report_{}.csv'
conf_filepath_format = 'confusion_matrix_{}.csv'
predictions_filepath_format = 'predictions_{}.csv'
target_dirs

['./results\\FiQA_k-folds',
 './results\\FinancialPhraseBank_DS50_k-folds',
 './results\\FinancialPhraseBank_DS66_k-folds',
 './results\\FinancialPhraseBank_DS75_k-folds',
 './results\\FinancialPhraseBank_DSAll_k-folds',
 './results\\SemEval_k-folds']

Average results for k-folds

In [3]:
def concat_dfs(df_filepaths, out_filepath):
    dfs = []
    for i, df_filepath in enumerate(df_filepaths):
        df = pd.read_csv(df_filepath, header=0)
        dfs.append(df)
    concat_df = pd.concat(dfs)
    concat_df.to_csv(out_filepath, index=False)
    print('Created {}'.format(out_filepath))
    return concat_df
    
for target_dir in target_dirs:
    names = [os.path.basename(item).split('_')[-1].replace('.csv', '') for item \
                   in glob(os.path.join(target_dir, 'fold=0', report_filepath_format.format('*')))]
    
    for name in names:
        report_filepaths = glob(os.path.join(target_dir, '*', report_filepath_format.format(name)))
        out_dir = target_dir
        average_classification_report(report_filepaths, os.path.join(out_dir, report_filepath_format.format(name)))
        
#         conf_filepaths = glob(os.path.join(target_dir, '*', conf_filepath_format.format(name)))
#         sum_confusion_matrix(conf_filepaths, os.path.join(out_dir, conf_filepath_format.format(name)))
        
#         df_filepaths = glob(os.path.join(target_dir, '*', predictions_filepath_format.format(name)))
#         concat_dfs(df_filepaths,os.path.join(out_dir, predictions_filepath_format.format(name)))

Created ./results\FiQA_k-folds\classification_report_AFINN.csv
Created ./results\FiQA_k-folds\classification_report_bert-base-uncased.csv
Created ./results\FiQA_k-folds\classification_report_lm.csv
Created ./results\FiQA_k-folds\classification_report_MPQA.csv
Created ./results\FiQA_k-folds\classification_report_roberta-base.csv
Created ./results\FiQA_k-folds\classification_report_Sen140.csv
Created ./results\FiQA_k-folds\classification_report_senti-dd.csv
Created ./results\FiQA_k-folds\classification_report_SentiStrength.csv
Created ./results\FiQA_k-folds\classification_report_SO-CAL.csv
Created ./results\FiQA_k-folds\classification_report_swn.csv
Created ./results\FiQA_k-folds\classification_report_textblob.csv
Created ./results\FiQA_k-folds\classification_report_vader.csv
Created ./results\FiQA_k-folds\classification_report_word2vec-google-news-300.csv
Created ./results\FinancialPhraseBank_DS50_k-folds\classification_report_AFINN.csv
Created ./results\FinancialPhraseBank_DS50_k-folds

# Results

In [4]:
dfs = []
model_names = list(set([os.path.basename(item).split('_')[-1].replace('.csv', '') for item in \
                        glob(os.path.join(result_dir, '*', 'classification_report_*.csv'))]))
for model_name in model_names:
    filepaths = sorted(glob(os.path.join(result_dir, '*', 'classification_report_{}.csv'.format(model_name))))
    
    records = []
    for filepath in filepaths:
        dataset_mode = os.path.basename(os.path.dirname(filepath)).split('_')[-2]

        result = pd.read_csv(filepath).set_index('Unnamed: 0')
        records.append(('_'.join([dataset_mode, 'Precision']), result.loc['weighted avg']['precision']))
        records.append(('_'.join([dataset_mode, 'Recall']), result.loc['weighted avg']['recall']))
        records.append(('_'.join([dataset_mode, 'F1-score']), result.loc['weighted avg']['f1-score']))
    dfs.append(pd.DataFrame(records, columns=['Dataset_Measure', model_name]))

final_result = reduce(lambda df1,df2: pd.merge(df1,df2,on='Dataset_Measure'), dfs)
filepath = os.path.join(result_dir, 'FPB_SemEval_results.csv')
final_result.to_csv(filepath, index=False)
print('Created {}'.format(filepath))
final_result

Created ./results\FPB_SemEval_results.csv


,Dataset_Measure,textblob,SO-CAL,MPQA,Sen140,swn,senti-dd,roberta-base,bert-base-uncased,word2vec-google-news-300,lm,AFINN,vader,SentiStrength
0,FiQA_Precision,0.7329,0.6421,0.7691,0.6081,0.6498,0.8594,0.4182,0.5356,0.7610,0.8334,0.7377,0.7153,0.6935
1,FiQA_Recall,0.2546,0.3716,0.3900,0.4863,0.3624,0.3899,0.6444,0.5502,0.7683,0.2890,0.4471,0.4679,0.2730
2,FiQA_F1-score,0.3459,0.4526,0.4980,0.4924,0.4418,0.4937,0.5064,0.5186,0.7413,0.3635,0.5403,0.5513,0.3327
3,DS50_Precision,0.5155,0.5161,0.5143,0.5345,0.4778,0.7137,0.8578,0.8487,0.7392,0.6147,0.6332,0.6028,0.5641
4,DS50_Recall,0.4852,0.4546,0.4567,0.2196,0.3969,0.7102,0.8523,0.8474,0.7466,0.6232,0.5897,0.5396,0.5713
5,DS50_F1-score,0.4953,0.4575,0.4630,0.1540,0.4107,0.7057,0.8523,0.8457,0.7328,0.5914,0.5960,0.5452,0.5644
6,DS66_Precision,0.5275,0.5248,0.5215,0.5514,0.4851,0.7433,0.8948,0.8827,0.7650,0.6337,0.6504,0.6194,0.5794
7,DS66_Recall,0.4968,0.4588,0.4631,0.2112,0.4044,0.7358,0.8826,0.8750,0.7741,0.6363,0.6054,0.5534,0.5849
8,DS66_F1-score,0.5070,0.4633,0.4708,0.1462,0.4194,0.7322,0.8844,0.8757,0.7622,0.6023,0.6130,0.5599,0.5795
9,DS75_Precision,0.5426,0.5444,0.5284,0.5500,0.4916,0.7898,0.9366,0.9222,0.8017,0.6507,0.6713,0.6409,0.6063


### DS100

In [6]:
DATA_DIR = './results'
relabel_dict = {'negative': '0', 'neutral': '1', 'positive': '2'}

dfs = []
model_names = list(set([os.path.basename(item).split('_')[-1].replace('.csv', '') for item in glob(os.path.join(DATA_DIR, '*', 'classification_report_*.csv'))]))
for model_name in model_names:
    filepaths = sorted(glob(os.path.join(DATA_DIR, '*DSAll*', 'classification_report_{}.csv'.format(model_name))))
    
    records = []
    for filepath in filepaths:
        dataset_mode = os.path.basename(os.path.dirname(filepath)).split('_')[-2]

        result = pd.read_csv(filepath).set_index('Unnamed: 0')
        for class_mode in ['positive', 'negative', 'neutral']:
            try: item = result.loc[class_mode]
            except: item = result.loc[relabel_dict[class_mode]]
            records.append(('_'.join([class_mode, 'Precision']), item['precision']))
            records.append(('_'.join([class_mode, 'Recall']), item['recall']))
            records.append(('_'.join([class_mode, 'F1-score']), item['f1-score']))
    dfs.append(pd.DataFrame(records, columns=['Dataset_Measure', model_name]))

final_result = reduce(lambda df1,df2: pd.merge(df1,df2,on='Dataset_Measure'), dfs)
filepath = os.path.join(DATA_DIR, 'FPB_DS100_results.csv')
final_result.to_csv(filepath, index=False)
print('Created {}'.format(filepath))
final_result

Created ./results\FPB_DS100_results.csv


,Dataset_Measure,textblob,SO-CAL,MPQA,Sen140,swn,senti-dd,roberta-base,bert-base-uncased,word2vec-google-news-300,lm,AFINN,vader,SentiStrength
0,positive_Precision,0.3439,0.3158,0.3026,0.1797,0.2284,0.8515,0.9422,0.7025,0.6886,0.6371,0.4574,0.3889,0.4465
1,positive_Recall,0.3931,0.6121,0.5208,0.1864,0.3132,0.6493,0.9506,0.8243,0.6959,0.1632,0.7298,0.7384,0.3922
2,positive_F1-score,0.3668,0.4163,0.3827,0.1827,0.2637,0.7361,0.9458,0.7562,0.6918,0.2596,0.5615,0.5085,0.4172
3,negative_Precision,0.3689,0.2282,0.3711,0.1782,0.2562,0.6351,0.8928,0.9029,0.7565,0.4184,0.4196,0.3479,0.3321
4,negative_Recall,0.4768,0.1475,0.3317,0.9466,0.4690,0.8493,0.9607,0.4210,0.3767,0.4004,0.3369,0.2255,0.3652
5,negative_F1-score,0.4143,0.1782,0.3497,0.2997,0.3310,0.7260,0.9234,0.5250,0.5008,0.4087,0.3694,0.2720,0.3471
6,neutral_Precision,0.6686,0.7041,0.6385,0.7959,0.6017,0.8704,0.9877,0.9372,0.8739,0.6852,0.8392,0.8058,0.7435
7,neutral_Recall,0.5850,0.4857,0.4629,0.0332,0.3988,0.8904,0.9663,0.9705,0.9653,0.9005,0.6686,0.5743,0.7650
8,neutral_F1-score,0.6238,0.5748,0.5366,0.0633,0.4790,0.8801,0.9766,0.9532,0.9173,0.7782,0.7439,0.6705,0.7539
